In [191]:
import pandas as pd 
import random
import numpy as np 
import matplotlib.pyplot as plt 

random.seed(42)  

%matplotlib inline

USDC = "USDC"

class Portal:
    def __init__(self, name):
        self.name = name
        self.investments = {}

    def add_vault(self, vault, ratio):
        if vault not in self.investments:
            self.investments[vault] = {
                'shares': 0,
                'ratio': ratio
            }

    def invest(self, vault, assets):
            if vault in self.investments:
                shares = vault.deposit(assets)
                self.investments[vault]['shares'] += shares            
                return shares            
            else:
                raise ValueError("Vault not found in investments")


    def value_position(self, vault):
        if vault in self.investments:
            assets = vault.convert_to_assets(self.investments[vault]['shares'])            
            return assets    



class ERC4626:
    def __init__(self, protocol, name, depositAsset, totalShares, totalAssets):
        self.protocol = protocol
        self.name = name
        self.depositAsset = depositAsset
        self.totalShares = totalShares
        self.totalAssets = totalAssets

    def deposit(self, assets):
        shares = self.convert_to_shares(assets)
        self.totalAssets += assets  
        self.totalShares += shares 
        return shares     

    def withdraw(self, shares):
        if shares > self.totalShares:
            raise ValueError("Insufficient shares")
        
        assets = self.convert_to_assets(shares)
        self.totalShares -= shares
        self.totalAssets -= assets    
        return assets

    def convert_to_shares(self, assets):
        a = assets
        b = self.totalAssets
        t = self.totalShares

        shares = a * t / b if b > 0 else a  # avoid division by zero
        return shares

    def convert_to_assets(self, shares):
        s = shares
        b = self.totalAssets
        t = self.totalShares

        assets = s * b / t if t > 0 else s
        return assets

    
    def seed(self):        
        starting_balance = random.uniform(10,100)
        self.deposit(starting_balance)        
        operations = random.randint(1, 10) 

        for i in range(operations):
            if i % 2 == 0:
                self.deposit(random.randint(1, 10))                
            else:
                self.earn_interest(random.uniform(0, 1))                         
    

    def earn_interest(self, percent):
        amount = self.totalAssets * percent / 100
        self.totalAssets += amount


def initialize_vaults(num_vaults):
    vaults = []
    for i in range(num_vaults):
        vault_name = f"Morpho Vault {chr(65 + i)}"  # A, B, C, ...
        vault = ERC4626("Morpho", vault_name, USDC, 0, 0)
        vault.seed()
        vaults.append(vault)
    return vaults

def create_portal(vaults, name, ratios):
    if sum(ratios) == 100:
        portal = Portal(name)
        for i in range(len(vaults)):
            portal.add_vault(vaults[i], ratios[i])
        
        return portal
    
    else:
        raise ValueError("Ratios do not sum to 100")  


vaults = initialize_vaults(3)
ratios = [40, 30, 30]
portal_1 = create_portal(vaults, "Morpho Vault", ratios)


# Test investment
portal_1.invest(vaults[0], 100)
portal_1.invest(vaults[1], 75)
portal_1.invest(vaults[2], 75)

# print state
print("\nOverall State:")
for vault in vaults:
    print(f"\n{vault.name}:")
    print(f"  Total Assets: {vault.totalAssets:.2f} {USDC}")
    print(f"  Total Shares: {vault.totalShares:.2f}")
    print(f"  Portal's Shares: {portal_1.investments[vault]['shares']:.2f}")
    print(f"  Portal's Ratio: {portal_1.investments[vault]['ratio']}%")
    portal_value = portal_1.value_position(vault)
    print(f"  Portal's Position Value: {portal_value:.2f} {USDC}")

total_portal_value = sum(portal_1.value_position(vault) for vault in vaults)
print(f"\nTotal Portal Value: {total_portal_value:.2f} {USDC}")



Overall State:

Morpho Vault A:
  Total Assets: 172.55 USDC
  Total Shares: 172.55
  Portal's Shares: 100.00
  Portal's Ratio: 40%
  Portal's Position Value: 100.00 USDC

Morpho Vault B:
  Total Assets: 118.27 USDC
  Total Shares: 117.48
  Portal's Shares: 74.50
  Portal's Ratio: 30%
  Portal's Position Value: 75.00 USDC

Morpho Vault C:
  Total Assets: 111.89 USDC
  Total Shares: 111.57
  Portal's Shares: 74.78
  Portal's Ratio: 30%
  Portal's Position Value: 75.00 USDC

Total Portal Value: 250.00 USDC
